In [7]:
import os
import numpy as np
from conllu import parse
from collections import defaultdict
import torch

sentences = parse(open('id_pud-ud-test-revised-1-0.conllu', 'r').read())

In [2]:
# Format change: Word_BERT_sentenceID_wordID
word2index = {}
idx=0
words = defaultdict(int)
for sentence in sentences:
    suffix = '_BERT_'+sentence.metadata['sent_id']
    for word in sentence:
        cur_suffix = suffix + '_' + str(word['id'])
        new_word = word['form'] + cur_suffix  
        word['form'] = new_word
        word2index[new_word]=idx
        idx+=1

In [4]:
import json
with open('word2index.json', 'w') as file:
    json.dump(word2index, file)

In [5]:
import json
with open('word2index.json', 'r') as file:
    new_d = json.load(file)

In [4]:
def extract_bert(sentence, model, tokz, save_to, word2index):
    os.makedirs(save_to)
    indexes = []
    subtokens = ['[CLS]']
    for word in sentence:
        indexes.append(len(subtokens))
        real_word = word['form'].split('_BERT_')[0]
        subtoken = tokz.tokenize(real_word)
        subtokens += subtoken
    subtokens += ['[SEP]']
    
    subtoken_ids = tokz.convert_tokens_to_ids(subtokens)
    segment_ids = [0] * len(subtokens)
    
    subtoken_ids_t = torch.tensor(subtoken_ids).unsqueeze(0)
    segment_ids_t = torch.tensor(segment_ids).unsqueeze(0)
    
    output, _ = model(input_ids=subtoken_ids_t, token_type_ids=segment_ids_t)
    output = output.view(len(subtoken_ids), model.config.hidden_size)
    indexes = torch.tensor(indexes)
    selected_output = torch.index_select(output, 0, indexes)
    
    assert len(indexes) == len(sentence) == selected_output.shape[0]
    for idx in range(len(sentence)):
        word = sentence[idx]['form']
        array = selected_output[idx].data.numpy()
        np.save(save_to+str(word2index[word]), array)

In [11]:
#Extract mBERT embedding
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-multilingual-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
save_to='data/mbert/'

for sentence in sentences:
    extract_bert(sentence, bert, tokenizer, save_to, word2index)

In [5]:
#Extract IndoBERT embedding
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('indolem/indobert-base-uncased', do_lower_case=True)
bert = BertModel.from_pretrained('indolem/indobert-base-uncased')
save_to='data/indobert/'

for sentence in sentences:
    extract_bert(sentence, bert, tokenizer, save_to, word2index)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [14]:
#Extract MalayBERT embedding, Only cased-malay-bert is available
from transformers import AlbertTokenizer, BertModel

tokenizer = AlbertTokenizer.from_pretrained('huseinzol05/bert-base-bahasa-cased', 
                unk_token = '[UNK]', pad_token='[PAD]', do_lower_case=False)
bert = BertModel.from_pretrained('huseinzol05/bert-base-bahasa-cased')
save_to='data/malaybert/'

for sentence in sentences:
    extract_bert(sentence, bert, tokenizer, save_to, word2index)

In [11]:
#create new folds
import glob
os.makedirs('data/folds', exist_ok=True)

def write(conlls, fname):
    f = open(fname, 'w')
    for conll in conlls:
        f.write(conll.serialize())
    f.close()
    
for file in glob.glob('folds/*'):
    sentences = parse(open(file).read())
    save_to = 'data/folds/'+file.split('/')[-1]  
    for sentence in sentences:
        suffix = '_BERT_'+sentence.metadata['sent_id']
        for word in sentence:
            cur_suffix = suffix + '_' + str(word['id'])
            new_word = word['form'] + cur_suffix  
            word['form'] = new_word
    write(sentences, save_to)